<a href="https://colab.research.google.com/github/kalebsampaco/Ejercicios-en-google-colab/blob/master/An%C3%A1lisis_de_comentarios_negativos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
import json
import codecs
import numpy as np
import pandas as pd
from keras.callbacks import ModelCheckpoint
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,Input,Bidirectional,GlobalMaxPool1D,Flatten,BatchNormalization,LeakyReLU
from keras.layers.convolutional import Conv1D,Conv2D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence,text
from keras import Model
from keras.models import load_model
import keras
import keras.backend as K
#LECTURA DE FICHEROS DE DATOS
df = pd.read_csv('/home/dhome/Descargas/bullyingV3/train.csv')
fichero=df.values.tolist()
salida_binaria=[]
texto_entrada=[]

#BINARIZACION DE ETIQUETAS
for i in range(0,len(fichero)):
	if(fichero[i][2]==1 or fichero[i][3]==1 or fichero[i][4]==1 or fichero[i][5]==1 or fichero[i][6]==1 or fichero[i][7]==1):
		salida_binaria.append(1)
	if(fichero[i][2]==0 and fichero[i][3]==0 and fichero[i][4]==0 and fichero[i][5]==0 and fichero[i][6]==0 and fichero[i][7]==0):
		salida_binaria.append(0)
	texto_entrada.append(fichero[i][1])

#VECTORIZACION DE ENTRADAS
caracteristicas_maximas=1000
longitud_maxima=500
tokenizer = text.Tokenizer(num_words=caracteristicas_maximas)
tokenizer.fit_on_texts(list(texto_entrada))
entrada_entrenamiento_vectorizada = tokenizer.texts_to_sequences(texto_entrada)
entrada_entrenamiento = sequence.pad_sequences(entrada_entrenamiento_vectorizada, maxlen=longitud_maxima)

salida_entrenamiento=np.array(salida_binaria)


ptos=salida_entrenamiento[salida_entrenamiento==1]
TP=len(ptos)/len(salida_entrenamiento)
def create_weighted_binary_crossentropy(zero_weight, one_weight):

    def weighted_binary_crossentropy(y_true, y_pred):

        b_ce = K.binary_crossentropy(y_true, y_pred)
        weight_vector = y_true * one_weight + (1. - y_true) * zero_weight
        weighted_b_ce = weight_vector * b_ce

        return K.mean(weighted_b_ce)

    return weighted_binary_crossentropy

#CREACION DE MODELO
tamano_embedding = 150

entrada_red = Input(shape=(longitud_maxima, ))
x = Embedding(caracteristicas_maximas, tamano_embedding)(entrada_red)
x = Bidirectional(LSTM(150, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dense(1, activation="sigmoid")(x)

modelo = Model(inputs=entrada_red, outputs=x)
modelo.compile(loss=create_weighted_binary_crossentropy(TP, 1 - TP),optimizer=keras.optimizers.Adam(0.0001),metrics=['binary_accuracy'])
modelo.summary()


#ENTRENAMIENTO
batch_size = 300
epochs = 30
checkpoint = ModelCheckpoint('comentarios.h5', monitor='val_loss', verbose=1, save_best_only=True,save_weights_only=True, mode='auto')
#modelo.fit(entrada_entrenamiento, salida_entrenamiento, batch_size=batch_size, epochs=epochs,callbacks=[checkpoint],validation_split=0.2,shuffle=True,verbose=1)
modelo.load_weights('comentarios.h5')
predicciones=modelo.predict(entrada_entrenamiento[:10000],verbose=1)

#EJEMPLOS DE COMENTARIOS

#COMENTARIO NO NEGATIVO
negativo=predicciones[predicciones<0.1]
print(negativo[0])
texto_interes=np.array(texto_entrada[:10000])
texto_interes=np.expand_dims(texto_interes,axis=1)
negativo=texto_interes[predicciones<0.1]
print(negativo[0])

#COMENTARIO NEGATIVO
positivo=predicciones[predicciones>0.9]
print(positivo[0])
texto_interes=np.array(texto_entrada[:10000])
texto_interes=np.expand_dims(texto_interes,axis=1)
positivo=texto_interes[predicciones>0.9]
print(positivo[0])


Using TensorFlow backend.


FileNotFoundError: ignored